# Laboratorium 1 - content-based recommender

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix

In [2]:
# tworzymy reprezentacje filmow jako wektorow cech - na podstawie gatunkow

genres = [
    '(no genres listed)', 
    'Action', 
    'Adventure', 
    'Animation', 
    'Children', 
    'Comedy', 
    'Crime', 
    'Documentary', 
    'Drama', 
    'Fantasy', 
    'Film-Noir', 
    'Horror', 
    'IMAX', 
    'Musical', 
    'Mystery', 
    'Romance', 
    'Sci-Fi', 
    'Thriller', 
    'War', 
    'Western'
]
genres_no = len(genres)

movies = pandas.read_csv('ml-latest-small/movies.csv')
movies_no = movies.shape[0]

movies['bias'] = 1.0
for genre in genres:
    movies[genre] = np.where(movies['genres'].str.contains(genre, regex=False), 1.0, 0.0)
    
movies = movies.drop(columns=['title', 'genres']).set_index('movieId')
movies

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# wczytujemy oceny uytkownikow i od razu dzielimy je na dwa zbiory - treningowy i testowy

all_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
train_ratings_set, test_ratings_set = train_test_split(all_ratings, test_size=0.05)
train_ratings_set

,userId,movieId,rating
86263,560,4023,2.5
51077,330,1573,0.5
46094,305,2329,5.0
54921,365,1704,5.0
20146,132,4963,3.0
...,...,...,...
1529,15,71264,2.5
79163,490,6365,2.5
18576,119,4387,4.5
74357,474,4874,2.5


In [4]:
# inicjalizujemy macierz preferencji uzytkownikow liczbami losowymi z przedzialu [0.0, 5.0]

def initialize_users(raw_ratings):
    users_no = raw_ratings['userId'].unique().size
    users = pandas.DataFrame(5.0 * np.random.uniform(size=(users_no, genres_no+1)), index=raw_ratings['userId'].unique(), columns=['bias']+genres)
    return users_no, users

users_no, users = initialize_users(train_ratings_set)
users

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
560,3.303611,4.854103,1.779426,1.880707,0.521565,3.579219,4.782040,4.713898,3.652190,2.367833,...,0.730360,1.772665,4.491385,2.680375,2.239260,1.886397,2.709790,4.045211,2.368507,1.704253
330,2.187501,2.287673,2.916812,4.743579,2.739938,0.899511,2.281560,2.579408,4.467094,1.592147,...,2.552589,0.513751,3.513037,1.895966,2.071049,2.370672,0.299053,2.652472,3.754100,2.747921
305,3.047951,4.626975,4.823521,3.800059,0.184912,1.222978,3.841869,1.857219,0.123231,3.851151,...,2.780986,3.729711,2.390970,0.882374,2.470657,1.047565,4.388646,4.700787,4.302262,4.890060
365,1.072140,3.051719,2.060657,3.411747,0.695251,2.803295,3.435749,0.005069,0.003262,4.304409,...,0.237396,3.705533,2.989837,2.380075,2.678718,4.299741,1.539806,0.599310,4.022724,2.011921
132,1.541211,1.592868,1.979320,1.665141,1.547277,3.918066,1.535374,0.057839,3.421318,1.950718,...,2.832040,2.262938,4.502195,2.534915,2.817330,1.137090,4.482861,2.212221,2.711447,3.353125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,3.675087,0.012412,1.589215,0.551758,0.139839,2.181299,2.497814,4.267645,3.270179,4.611282,...,1.104339,3.223461,2.673832,3.261619,0.662858,4.448483,4.116338,1.799749,0.022403,2.226057
120,2.842294,2.483794,1.629056,1.825011,0.387940,3.625383,2.008250,2.961387,0.753983,2.260708,...,3.152639,4.141668,0.460172,3.360242,2.713355,3.402995,3.986103,1.996460,3.873879,4.618695
278,2.876673,3.624099,0.901864,3.975479,2.900133,1.004937,0.660375,1.738851,1.084881,0.795838,...,1.310301,2.140874,3.124592,0.340132,0.794539,3.082003,0.197861,2.325244,4.702607,4.262118
459,2.542682,0.678070,4.256480,4.362560,1.251772,0.357794,4.133599,3.271084,4.738058,4.884658,...,3.248413,4.381447,1.458199,4.545297,1.023592,0.860277,4.832586,3.056095,2.153024,1.725864


In [5]:
# za pomoca sprytnej sztuczki przeksztalcamy oceny z formatu dostarczonego przez MovieLens do uzytecznej macierzy
# zwroc uwage na to, ze czesci filmow moze brakowac po podziale datasetu na dwie czesci - musimy uzueplnic brakujace kolumny

def get_ratings(raw_ratings, movies):
    ratings = raw_ratings.pivot(*raw_ratings.columns).fillna(0.0)
    missing_movies = set(movies.index).difference(set(raw_ratings['movieId']))
    for movie in missing_movies:
        ratings[movie] = 0.0
    ratings = ratings.reindex(sorted(ratings.columns), axis=1)
    return ratings

ratings = get_ratings(train_ratings_set, movies)
ratings

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_11612/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Część 2. - trening modelu

In [6]:
# trenujemy model iteracyjnie, wykorzystujac gradient descent

alpha = 0.000001 # learning speed
delta = 100 # minimal upgrade for each step
lambd = 0.001 # regularization weight

def calculate_user_preferences(users, movies, ratings, raw_ratings, users_no, movies_no, alpha, delta, lambd):
    total_error = 0.0
    model = users

    while(True):
        previous_total_error = total_error

        predicted_ratings = np.dot(model, movies.T) # mozemy to policzyc jako iloczyn skalarny preferencji uzytkownikow i cech filmow
        # tu stosujemy bardzo przydatna funkcje NumPy
        errors = np.where(ratings==0.0, 0, predicted_ratings - ratings)
        errors
        gradient = errors.dot(movies) # znow iloczyn skalarny - tym razem bledow

        # tu stosujemy pewna sztuczke - rozbijamy sobie macierz z wyrazami regularyzujacymi na dwie
        # pierwsza to kolumna zlozona z zer
        regularization_k0 = pandas.DataFrame(np.zeros((users_no, 1)), index=raw_ratings['userId'].unique(), columns=['bias'])
        # druga to macierz preferencji uzytkownikow (czyli modelu) - bez pierwszej kolumny
        regularization_k = model.drop(columns=['bias'])
        # teraz sklejamy obie macierze
        regularization = pandas.concat([regularization_k0, regularization_k], axis=1)

        # najwazniejszy krok - aktualizacja modelu, czyli wszystkich wag
        # na wykładzie było +lambda ale wtedy się krzaczy, ale może normalnie lambda jest ujemna
        model = model - alpha * (gradient - lambd * regularization) 

        total_error = np.sum(errors**2)
        print(total_error)
        progress = abs(previous_total_error - total_error)
        if progress < delta:
            break
            
    return model

prediction_model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)
prediction_model

5240400.819138352
5231456.563200748
5222549.027578918
5213677.997936251
5204843.261398718
5196044.606544253
5187281.823391997
5178554.703391763
5169863.039413545
5161206.625737073
5152585.258041455
5143998.733394992
5135446.850244845
5126929.408407154
5118446.209056743
5109997.054717397
5101581.749251847
5093200.09785201
5084851.90702927
5076536.984604824
5068255.139700089
5060006.18272717
5051789.925379504
5043606.18062242
5035454.762683856
5027335.487045217
5019248.170432105
5011192.630805314
5003168.687351839
4995176.160475831
4987214.871789806
4979284.644105861
4971385.301426809
4963516.668937657
4955678.572996863
4947870.841127867
4940093.302010606
4932345.785473073
4924628.122482991
4916940.145139509
4909281.686664987
4901652.581396845
4894052.6647794135
4886481.773355958
4878939.744760657
4871426.417710722
4863941.631998472
4856485.228483619
4849057.049085502
4841656.936775345
4834284.7355687255
4826940.290517955
4819623.447704599
4812334.054231992
4805071.958217875
4797837.0087

3147348.5222437056
3144872.709967065
3142401.775769195
3139935.703786706
3137474.478228954
3135018.083377624
3132566.5035863034
3130119.723280081
3127677.726955152
3125240.4991783695
3122808.0245868876
3120380.287887691
3117957.2738573006
3115538.967341261
3113125.3532538353
3110716.416577533
3108312.142362834
3105912.5157276792
3103517.521857161
3101127.146003104
3098741.3734837435
3096360.18968326
3093983.5800514836
3091611.5301034898
3089244.0254192157
3086881.051643126
3084522.5944838435
3082168.639713755
3079819.173168663
3077474.180747473
3075133.6484117806
3072797.562185553
3070465.9081547447
3068138.672466992
3065815.8413312496
3063497.4010174233
3061183.337856069
3058873.6382380323
3056568.288614089
3054267.2754946696
3051970.585449468
3049678.2051071264
3047390.121154949
3045106.3203384974
3042826.7894613435
3040551.5153847
3038280.4850271065
3036013.6853641397
3033751.103428068
3031492.72630753
3029238.5411472647
3026988.5351477712
3024742.6955649843
3022501.00971002
3020263

2373576.072806785
2372344.6406315533
2371114.7796623157
2369886.4867044124
2368659.7585722553
2367434.592089198
2366210.9840875873
2364988.931408687
2363768.43090265
2362549.479428508
2361332.0738540953
2360116.211056056
2358901.8879198004
2357689.101339467
2356477.8482178845
2355268.1254665656
2354059.930005639
2352853.258763858
2351648.1086785244
2350444.476695498
2349242.3597691446
2348041.754862304
2346842.6589462664
2345645.069000741
2344448.9820138216
2343254.3949819533
2342061.3049099133
2340869.7088107676
2339679.6037058495
2338490.986624729
2337303.8546051807
2336118.2046931563
2334934.0339427516
2333751.3394161654
2332570.1181837115
2331390.367323742
2330212.0839226414
2329035.265074799
2327859.907882578
2326686.009456271
2325513.5669140983
2324342.5773821594
2323173.037994413
2322004.9458926492
2320838.2982264576
2319673.0921532125
2318509.3248380087
2317346.993453685
2316186.0951807606
2315026.627207423
2313868.5867294846
2312711.9709503907
2311556.777081138
2310403.0023403

1946713.9160626621
1945958.079655241
1945202.9662234453
1944448.5746667618
1943694.9038869762
1942941.952788142
1942189.7202765876
1941438.205260906
1940687.406651949
1939937.323362822
1939187.9543088768
1938439.2984077067
1937691.3545791453
1936944.121745254
1936197.598830311
1935451.7847608228
1934706.6784654988
1933962.278875271
1933218.584923246
1932475.5955447517
1931733.3096772905
1930991.726260559
1930250.8442364102
1929510.6625489031
1928771.180144227
1928032.395970762
1927294.308979025
1926556.9181216916
1925820.222353581
1925084.2206316406
1924348.9119149696
1923614.2951647795
1922880.3693444035
1922147.1334193088
1921414.5863570513
1920682.7271273055
1919951.5547018426
1919221.068054533
1918491.2661613252
1917762.148000268
1917033.7125514694
1916305.9587971314
1915578.8857215098
1914852.4923109196
1914126.7775537458
1913401.7404404231
1912677.3799634257
1911953.6951172715
1911230.6848985138
1910508.348305747
1909786.6843395748
1909065.6920026368
1908345.3702995745
1907625.71

1672354.765588751
1671834.9282332289
1671315.488675009
1670796.446427108
1670277.8010033562
1669759.5519184072
1669241.6986877127
1668724.2408275476
1668207.17785499
1667690.509287927
1667174.2346450428
1666658.353445837
1666142.8652106046
1665627.7694604392
1665113.0657172345
1664598.7535036863
1664084.8323432778
1663571.3017602903
1663058.1612797931
1662545.4104276414
1662033.048730502
1661521.0757157966
1661009.4909117576
1660498.293847378
1659987.484052452
1659477.061057554
1658967.024394013
1658457.373593968
1657948.1081903058
1657439.2277167095
1656930.7317076095
1656422.619698231
1655914.891224558
1655407.5458233329
1654900.5830320779
1654394.0023890713
1653887.8034333605
1653381.985704741
1652876.548743785
1652371.492091805
1651866.815290883
1651362.5178838545
1650858.5994142953
1650355.0594265508
1649851.8974656996
1649349.1130775784
1648846.7058087706
1648344.6752066019
1647843.0208191474
1647341.7421952079
1646840.8388843434
1646340.3104368509
1645840.1564037486
1645340.3763

...

1041465.4426077402
1041302.6355300199
1041139.890710612
1040977.2081117396
1040814.5876956593
1040652.0294246555
1040489.5332610499
1040327.0991671882
1040164.727105451
1040002.4170382514
1039840.1689280347
1039677.9827372709
1039515.8584284715
1039353.7959641715
1039191.7953069389
1039029.8564193749
1038867.9792641076
1038706.1638038041
1038544.4100011524
1038382.7178188805
1038221.0872197415
1038059.5181665224
1037898.0106220385
1037736.5645491396
1037575.1799107074
1037413.8566696472
1037252.5947889026
1037091.3942314408
1036930.2549602704
1036769.17693842
1036608.1601289554
1036447.2044949706
1036286.3099995891
1036125.4766059672
1035964.7042772918
1035803.992976778
1035643.3426676769
1035482.75331326
1035322.2248768419
1035161.7573217546
1035001.3506113729
1034841.0047090901
1034680.7195783433
1034520.4951825865
1034360.3314853077
1034200.2284500357
1034040.186040311
1033880.2042197221
1033720.2829518773
1033560.4222004157
1033400.6219290075
1033240.8821013576
1033081.2026811927
1

974662.2001732559
974523.7777085613
974385.4033795521
974247.0771597043
974108.7990225237
973970.5689415307
973832.3868902673
973694.2528422999
973556.1667712043
973418.1286505858
973280.1384540647
973142.1961552872
973004.3017279111
972866.45514562
972728.6563821142
972590.9054111169
972453.2022063682
972315.5467416309
972177.9389906845
972040.3789273315
971902.8665253903
971765.4017587019
971627.9846011264
971490.6150265444
971353.2930088536
971216.0185219756
971078.7915398463
970941.6120364271
970804.4799856935
970667.3953616432
970530.358138296
970393.3682896853
970256.4257898688
970119.5306129261
969982.6827329426
969845.8821240407
969709.1287603537
969572.4226160342
969435.763665257
969299.1518822107
969162.5872411107
969026.0697161873
968889.5992816921
968753.1759118915
968616.7995810779
968480.4702635594
968344.187933664
968207.9525657337
968071.764134142
967935.6226132712
967799.5279775274
967663.4802013309
967527.4792591294
967391.5251253818
967255.6177745702
967119.757181197

915913.8155158875
915794.9515667588
915676.1254366207
915557.3371064328
915438.5865571658
915319.8737698057
915201.1987253458
915082.5614047991
914963.9617891886
914845.3998595509
914726.8755969311
914608.3889824001
914489.9399970281
914371.5286219044
914253.1548381321
914134.8186268244
914016.5199691096
913898.2588461278
913780.0352390358
913661.8491289965
913543.7004971908
913425.5893248142
913307.5155930672
913189.479283171
913071.4803763595
912953.5188538745
912835.5946969732
912717.7078869257
912599.8584050165
912482.0462325388
912364.2713508075
912246.5337411357
912128.8333848651
912011.1702633371
911893.5443579156
911775.9556499749
911658.4041208948
911540.8897520795
911423.4125249382
911305.9724208933
911188.5694213853
911071.2035078597
910953.8746617805
910836.5828646233
910719.3280978767
910602.110343037
910484.9295816192
910367.785795151
910250.678965164
910133.6090732187
910016.5761008717
909899.5800297017
909782.6208412973
909665.6985172596
909548.8130392032
909431.9643887

865047.7811293595
864944.4658937976
864841.1809542687
864737.9262967224
864634.7019071175
864531.5077714289
864428.3438756301
864325.2102057092
864222.1067476632
864119.0334874954
864015.9904112236
863912.9775048668
863809.9947544602
863707.0421460399
863604.1196656595
863501.2272993724
863398.3650332511
863295.5328533686
863192.7307458101
863089.9586966702
862987.2166920501
862884.5047180594
862781.8227608216
862679.1708064625
862576.5488411223
862473.9568509444
862371.3948220887
862268.8627407115
862166.3605929944
862063.88836511
861961.4460432575
861859.0336136259
861756.651062433
861654.298375886
861551.9755402154
861449.6825416556
861347.4193664476
861245.1860008397
861142.9824310924
861040.8086434801
860938.6646242732
860836.5503597616
860734.4658362366
860632.4110400049
860530.3859573804
860428.3905746733
860326.4248782229
860224.4888543631
860122.5824894371
860020.7057698073
859918.858681831
859817.0412118844
859715.2533463448
859613.4950716038
859511.7663740597
859410.06724011

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
560,-0.181702,4.854130,0.523795,0.676154,-0.055784,2.648433,2.992674,3.435682,3.652209,1.370276,...,0.698941,1.491735,4.491410,2.261492,1.688835,1.471048,2.065872,2.770550,2.123787,1.622640
330,1.585126,2.287685,2.506483,4.606499,2.739953,0.899516,2.200455,2.232288,4.422586,1.213920,...,2.552603,0.490273,3.328440,1.895977,1.982153,2.364536,0.179004,2.361800,3.725783,2.719632
305,1.217222,4.627000,3.974339,3.166830,0.001804,0.991692,3.391889,1.711792,0.123232,3.043999,...,2.781002,3.378985,2.390983,0.825881,2.408646,0.814459,3.594395,4.207833,3.976625,4.890087
365,-0.984559,3.051735,1.639209,2.796716,0.502558,2.534810,2.154395,0.026618,0.032169,2.663198,...,0.252156,3.647336,2.940085,2.003242,2.439549,2.888054,1.393202,0.335053,3.783919,1.914862
132,0.871928,1.592877,1.785568,1.408375,1.292208,3.602140,1.335162,-0.023870,3.421337,1.630217,...,2.832055,2.250896,4.333601,2.279851,2.805635,0.931057,4.420000,2.092765,2.640035,3.288966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,0.324055,0.012412,0.424784,-0.035982,-0.103018,1.694417,0.993423,2.466690,3.242184,2.018767,...,1.055622,2.583041,2.408553,2.613035,0.107007,1.544294,2.943669,0.263119,0.118915,2.036933
120,0.021069,2.483807,0.512507,0.888333,0.174463,2.953956,0.882418,2.533020,0.753987,1.317818,...,3.152656,3.389161,0.463695,3.177368,2.349419,2.787165,3.087355,0.940711,3.773306,4.582407
278,0.713580,3.624119,-0.103601,1.717361,1.789309,-0.284253,0.024368,1.443714,1.117482,0.572621,...,1.241311,1.528778,2.840120,-0.080341,0.365860,1.994744,0.006429,1.343965,4.400299,4.009374
459,1.133979,0.678073,3.653444,3.862869,1.203077,0.291572,3.853651,3.015617,4.699830,4.125942,...,3.248431,4.313535,1.410239,4.545322,1.023597,0.703426,4.507353,2.409532,1.985440,1.681881


## Część 3. - ocena jakości algorytmu

In [7]:
# na podstawie zbioru testowego i wytrenowanego modelu obliczamy metryki opisujace jakosc modelu

positive_threshold = 4.0
negative_threshold = 2.0

# Funkcja pomocnicza zamieniająca ratingi trójwartościową informację: -1 jeśli poniżej negatywnego tresholda(nie polecać), 
# 1 jeśli powyżej pozytywnego(polecać) i 0 jeśli pomiędzy tresholdami(nie wiadomo)
def classify_ratings(ratings, p_treshold, n_treshold):
    #zrobione na podstawie https://numpy.org/doc/stable/reference/generated/numpy.where.html
    return np.where(ratings <= n_treshold, -1, np.where(ratings < p_treshold, 0, 1))

def calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold):
    # zamieniamy ratingi na informacje bardziej przystępną dla sklearnowego confusion matrix
    
    #tu normalizujemy ratingi (ucinamy id userów i filmów, bo sklearnowe confusion matrix ich nie potrzebuje)
    test_set_classified = classify_ratings(test_ratings_set['rating'], positive_threshold, negative_threshold)
    
    # to pewnie można zrobić dużo lepiej, ale zmęczenie wzięło górę
    #inicjalizuje tablicę długości o tej samej długości co testowa
    predicted_set_classified = np.zeros(test_ratings_set['rating'].size)
    # mapuję userId i movieId dla wszystkich wpisów w zbiorze testowym na na predykcje modelu
    # (dla każdego rzędu biorę movieId i userId i z macierzy predykcji pobieram predykcję dla tego movieId i userId)
    iterator = 0
    for index, row in test_ratings_set.iterrows():
        predicted_set_classified[iterator] = predicted_ratings[row['movieId']][ row['userId']]
        iterator+=1
    # tu znowu funkcja która zamienia nam ratingi na informację czy polecać, czy nie polecać, czy nie wiadomo
    predicted_set_classified = classify_ratings(predicted_set_classified, positive_threshold, negative_threshold)
    
    # nastepnie wyliczam wszystkie metryki
    # ustawiam sobię labelki(labels=[1, -1, 0]) tak żeby oceny neutralne były na końcu i je odcinam odcinam([0:2, 0:2]) 
    # w efekcie dostaję macierz konfuzji 2 na 2
    conf_matrix = confusion_matrix(test_set_classified, predicted_set_classified, labels=[1, -1, 0])[0:2, 0:2]
    
    true_positives = conf_matrix[0, 0]
    false_positives = conf_matrix[0, 1]
    true_negatives = conf_matrix[1, 0]
    false_negatives = conf_matrix[1, 1]
    
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2*precision*recall/( precision + recall )
        
    return {
        'true_positives': true_positives,
        'true_negatives': true_negatives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [8]:
predicted_ratings = prediction_model.dot(movies.T)

calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold)

{'true_positives': 1721,
 'true_negatives': 482,
 'false_positives': 206,
 'false_negatives': 65,
 'accuracy': 0.8904607922392886,
 'precision': 0.8930980799169694,
 'recall': 0.9636058230683091,
 'f1': 0.9270131968758417}

In [9]:
# dla porownania - obliczmy te same metryki dla modelu losowego
# zauwaz, w jaki sposob ponownie wykorzystujemy funkcje inicjalizujaca preferencje uzytkownikow

_, random_model = initialize_users(train_ratings_set)
random_prediction = random_model.dot(movies.T)
calculate_stats(test_ratings_set, random_prediction, positive_threshold, negative_threshold)

{'true_positives': 2222,
 'true_negatives': 597,
 'false_positives': 28,
 'false_negatives': 11,
 'accuracy': 0.9863540937718684,
 'precision': 0.9875555555555555,
 'recall': 0.9950738916256158,
 'f1': 0.9913004684363149}

## Część 4. - istotność statystyczna

In [ ]:
# wielokrotnie uruchamiamy trening modelu
# za każdym razem dzielimy dataset na zbior treningowy i testowy w inny sposob - klasa KFold robi to za nas
# zwroc uwage na bardzo istotny szczegol - oba modele, wytrenowany i losowy, musza byc porownywane na tym samym zbiorze testowym

n_tests = 5
results = []
random_results = []

for train, test in KFold(n_splits=n_tests, shuffle=True).split(all_ratings):
    training_set = all_ratings.iloc[train]
    test_set = all_ratings.iloc[test]
    # wygeneruj macierz użytkowników i ocen
    users_no, users = initialize_users(training_set)
    ratings = get_ratings(training_set, movies)
    # wytrenuj model
    model = calculate_user_preferences(users, movies, ratings, training_set, users_no, movies_no, alpha, delta, lambd)
    model_preferences = model.dot(movies.T)
    
    _, random_model = initialize_users(training_set)
    
    random_preferences = random_model.dot(movies.T)
    # oblicz metryki dla wytrenowanego modelu
    results.append(calculate_stats(test_set, model_preferences, positive_threshold, negative_threshold))
    # oblicz metryki dla modelu losowego
    random_results.append(calculate_stats(test_set, random_preferences, positive_threshold, negative_threshold))
    


C:\Users\Lenovo\AppData\Local\Temp/ipykernel_11612/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


3956048.41654301
3950817.434127392
3945602.536964139
3940403.654630512
3935220.7170779463
3930053.6546297697
3924902.397979087
3919766.878186482
3914647.0266778977
3909542.775242425
3904454.0560301635
3899380.8015500936
3894322.944667901
3889280.418603905
3884253.1569309174
3879241.0935722054
3874244.1627993407
3869262.2992302096
3864295.437826897
3859343.51389373
3854406.463075125
3849484.221353741
3844576.7250483604
3839683.9108119137
3834805.715629577
3829942.076816725
3825092.932017042
3820258.219200559
3815437.876661751
3810631.8430175777
3805840.057205662
3801062.4584823074
3796298.9864207073
3791549.580909038
3786814.1821486023
3782092.7306520236
3777385.167241374
3772691.4330463833
3768011.469502634
3763345.2183497595
3758692.6216296614
3754053.62168474
3749428.1611561375
3744816.1829819577
3740217.6303956173
3735632.4469239763
3731060.5763857607
3726501.9628897724
3721956.550833231
3717424.2849000497
3712905.1100592236
3708398.9715631134
3703905.8149458193
3699425.5860215244
3

2581716.766706636
2579888.080196001
2578062.710254245
2576240.647478052
2574421.8824994084
2572606.405985487
2570794.2086384385
2568985.2811952545
2567179.6144275996
2565377.1991416556
2563578.026177974
2561782.0864112996
2559989.3707504175
2558199.870138029
2556413.5755505357
2554630.4779979573
2552850.568523731
2551073.838204568
2549300.278150314
2547529.8795037777
2545762.6334406105
2543998.531169133
2542237.5639302023
2540479.7229970484
2538724.99967515
2536973.385302065
2535224.8712473065
2533479.448912174
2531737.109729647
2529997.845164195
2528261.6467116755
2526528.50589918
2524798.4142848733
2523071.363457893
2521347.345038171
2519626.350676337
2517908.3720535226
2516193.4008812793
2514481.428901433
2512772.4478859087
2511066.4496366438
2509363.425985437
2507663.3687937874
2505966.26995282
2504272.1213830994
2502580.915034528
2500892.6428861925
2499207.2969462783
2497524.869251875
2495845.351868904
2494168.736891971
2492495.016444228
2490824.1826772518
2489156.2277709297
24874

2001667.110501404
2000720.6011519057
1999775.2559241848
1998831.0725079633
1997888.048599195
1996946.181899996
1996005.4701186554
1995065.910969591
1994127.5021733355
1993190.2414565335
1992254.1265519005
1991319.1551982153
1990385.325140278
1989452.6341289252
1988521.0799209832
1987590.6602792495
1986661.3729724942
1985733.2157754074
1984806.1864686126
1983880.2828386177
1982955.5026778255
1982031.8437844887
1981109.3039626877
1980187.8810223562
1979267.5727791952
1978348.3770547062
1977430.291676151
1976513.314476536
1975597.4432945873
1974682.6759747476
1973769.0103671285
1972856.4443275372
1971944.9757174056
1971034.6024038119
1970125.3222594394
1969217.133162579
1968310.0329970769
1967404.0196523522
1966499.091023367
1965595.245010595
1964692.4795200108
1963790.7924630912
1962890.1817567584
1961990.6453234025
1961092.1810908392
1960194.786992292
1959298.4609663915
1958403.200957144
1957509.004913906
1956615.870791403
1955723.7965496602
1954832.7801540284
1953942.819575147
1953053.

1673043.6843893633
1672449.5078146772
1671855.8764371737
1671262.78943564
1670670.2459905695
1670078.2452841387
1669486.7865002414
1668895.868824433
1668305.491443967
1667715.6535477692
1667126.3543264433
1666537.5929722597
1665949.3686791556
1665361.6806427334
1664774.5280602493
1664187.9101306081
1663601.8260543747
1663016.275033743
1662431.2562725577
1661846.7689763042
1661262.8123520839
1660679.3856086342
1660096.487956315
1659514.118607108
1658932.2767746057
1658350.9616740127
1657770.172522141
1657189.9085374
1656610.1689398077
1656030.9529509663
1655452.2597940704
1654874.0886938993
1654296.438876827
1653719.3095707882
1653142.7000052966
1652566.6094114438
1651991.0370218745
1651415.9820708055
1650841.443794008
1650267.4214288024
1649693.9142140676
1649120.921390221
1648548.4421992302
1647976.4758845852
1647405.0216913312
1646834.0788660243
1646263.6466567623
1645693.7243131523
1645124.3110863292
1644555.4062289358
1643987.0089951358
1643419.1186405863
1642851.7344224593
1642284

...

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_11612/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


4101899.402196301
4096472.814279568
4091062.9418068016
4085669.712793382
4080293.055621633
4074932.8990388247
4069589.172155104
4064261.804441466
4058950.7257277044
4053655.8662004652
4048377.156401207
4043114.5272242287
4037867.909914696
4032637.236066678
4027422.4376212177
4022223.4468643614
4017040.1964252726
4011872.619274267
4006720.6487209643
4001584.2184123746
3996463.2623309847
3991357.714792936
3986267.5104461526
3981192.5842684815
3976132.8715658584
3971088.307970501
3966058.8294390803
3961044.372250883
3956044.873006088
3951060.2686239285
3946090.4963409444
3941135.4937092056
3936195.198594569
3931269.5491749374
3926358.48393852
3921461.9416821273
3916579.861509422
3911712.1828292874
3906858.8453540676
3902019.7890979187
3897194.954375153
3892384.281798514
3887587.712277615
3882805.1870172196
3878036.647515669
3873282.0355632002
3868541.2932403926
3863814.3629165245
3859101.1872479906
3854401.709176717
3849715.8719285866
3845043.6190118953
3840384.894215719
3835739.641608485

2682202.5823655706
2680313.9268067237
2678428.702752238
2676546.9003070397
2674668.5096137715
2672793.5208526156
2670921.924241159
2669053.7100341795
2667188.868523508
2665327.390037841
2663469.2649425673
2661614.4836396356
2659763.0365673457
2657914.9142002095
2656070.1070487676
2654228.6056594457
2652390.4006143813
2650555.482531257
2648723.8420631518
2646895.469898369
2645070.356760278
2643248.493407156
2641429.8706320343
2639614.479262548
2637802.3101607575
2635993.354223004
2634187.60237977
2632385.0455954997
2630585.674868468
2628789.4812306063
2626996.4557473576
2625206.58951755
2623419.8736732122
2621636.2993794344
2619855.8578342376
2618078.5402684077
2616304.3379453383
2614533.242160914
2612765.2442433387
2611000.3355530193
2609238.507482382
2607479.7514557648
2605724.058929244
2603971.4213905307
2602221.830358779
2600475.277384512
2598731.754049413
2596991.2519662166
2595253.762778595
2593519.2781609735
2591787.78981843
2590059.289486532
2588333.7689312245
2586611.219948686


2081474.932688518
2080494.5672800625
2079515.391420815
2078537.402753386
2077560.598926819
2076584.9775965519
2075610.5364244073
2074637.273078573
2073665.1852335744
2072694.270570246
2071724.5267757138
2070755.9515433984
2069788.5425729442
2068822.2975702465
2067857.2142473995
2066893.290322695
2065930.523520581
2064968.9115716587
2064008.4522126468
2063049.1431863778
2062090.9822417644
2061133.9671337728
2060178.095623429
2059223.3654777633
2058269.774469819
2057317.3203786183
2056366.0009891388
2055415.8140923043
2054466.7574849543
2053518.8289698348
2052572.0263555704
2051626.3474566375
2050681.7900933686
2049738.3520919075
2048796.0312841993
2047854.825507979
2046914.732606736
2045975.7504297108
2045037.8768318663
2044101.1096738514
2043165.4468220402
2042230.8861484297
2041297.4255306995
2040365.0628521363
2039433.796001649
2038503.622873728
2037574.5413684445
2036646.5493914217
2035719.6448538115
2034793.825672292
2033869.0897690332
2032945.435071697
2032022.859513384
2031101.36

1740615.8651733967
1739995.7523660082
1739376.2011349024
1738757.2106455273
1738138.7800650545
1737520.908562375
1736903.5953080854
1736286.839474514
1735670.6402356622
1735054.9967672455
1734439.9082466925
1733825.373853096
1733211.3927672529
1732597.9641716338
1731985.0872503906
1731372.7611893513
1730760.9851760135
1730149.7583995285
1729539.0800507227
1728928.949322073
1728319.365407712
1727710.327503413
1727101.8348065896
1726493.8865163128
1725886.481833268
1725279.6199597823
1724673.3000998092
1724067.521458919
1723462.2832443085
1722857.5846647737
1722253.4249307304
1721649.8032542067
1721046.7188488208
1720444.1709297772
1719842.1587139042
1719240.6814195977
1718639.738266836
1718039.3284771917
1717439.4512738024
1716840.1058813836
1716241.2915262165
1715643.0074361498
1715045.2528405897
1714448.0269705025
1713851.3290584004
1713255.158338346
1712659.5140459521
1712064.3954183715
1711469.801694282
1710875.7321139122
1710282.1859189982
1709689.1623528122
1709096.6606601495
1708

In [ ]:
# obliczamy, w ilu probach wytrenowany model okazal sie lepszy od losowego
# przeprowadzamy test statystyczny - jak prawdopodobne jest to, by k pozytywnych prob bylo dzielem przypadku

def possibility_of_at_least_k_successes_in_n(k, n):
    p = 0.0
    # obliczamy kolejno prawdopodobienstwo k sukcesow, k+1 sukcesow, ...
    # przydadza Ci sie funkcje marh.comb() i math.pow()
    for i in range(k, n+1):
        p += math.comb(n, i) * math.pow(0.5, i) * math.pow(0.5, n-i)
    
    return p

p = 0.05
metric = 'recall'

positive_tests_count = np.sum(list(map(lambda x, y: x[metric]>y[metric], results, random_results)))# w ilu przypadkach okazalismy sie lepsi niz random?


if possibility_of_at_least_k_successes_in_n(positive_tests_count, n_tests) <= p:
    print('We are better than random!')
else:
    print('There is no evidence we are better')